### IDX: decorator를 이해하기 위해서는..
- 동적 타입 언어
- First Class Function
- Closure


### Dynamic Programming Language(동적 타입 언어)

다른 언어에서 컴파일 과정 중 수행하는 작업을 런타임에 수행하는 고급언어.

즉, 런타임에 코드 추가, 타입 시스템 변경 등을 수행한다

>*파이썬은 [JIT 컴파일](https://namu.wiki/w/JIT)  하지 않는 dynamic interpreter 언어*


### First Class Function 
- 프로그래밍 언어가 함수(function)를 일급 객체(first-class citizen)으로 취급하는 것을 의미
- 함수도 데이터(객체)에 해당
    - 함수를 변수에 __할당__ 하거나 데이터 구조안에 저장 가능
    - 함수를 __매개변수__ 로써 다른 함수에 전달 가능
    - 함수를 결과값으로  __리턴__ 하는 함수 정의 가능
    - __동적__ 으로 프로퍼티 할당 가능
    

함수 hello를 정의 한 뒤 변수에 할당해보자

In [2]:
def hello(x):
    print('Hi i am a function')
    
firs = hello
print(id(firs) == id(hello))

True


동일한 주소값에 저장된 함수 객체가 firs 와 hello에 할당됨을 알 수 있다

따라서 firs로도 호출이 가능하다

In [7]:
print(callable(firs))
firs(12)

True
Hi i am a function


내장함수도 마찬가지이다

In [3]:
Ankit = print
Ankit("Hi I'm print")

Hi I'm print


함수 호출해서 리스트의 원소로 넣기도 가능

이 과정에서 return이 미뤄지는 일종의 lazy evaluation 발생


In [26]:
def f1():
    print('look')
    return 0
def f2():
    print('hello')
    return 0
def f3(r):
    print('oh', r)
    return 0

fs = [f1(),f2(),f3(12)]

for f in fs:
    print(f)

look
hello
oh 12
0
0
0


또한 함수의 인자로 다른 함수를 갖는 함수를 정의 할 수 있다

매개변수로 add_two 함수를 갖는 calculate 함수의 실행 결과를 살펴보자

In [11]:
def add_two(a, b):
    return a + b

def calculate(func, arg1, arg2):
    print('calculation:', func.__name__)
    print('result:', func(arg1, arg2))

calculate(add_two, 4, 10)

calculation: add_two
result: 14


다른 함수를 리턴하는 함수의 정의 역시 가능하다

hi 함수를 결과값으로 내어놓은 hello 함수를 정의해보자

In [18]:
def hi(x):
    print('hi')
    return

def hello(x):
    print('hello')
    return hi

hello(1)

hello


<function __main__.hi>

함수의 결과 값으로 __함수 자신__ 을 내어놓는 함수는 어떨까

In [19]:
def hello(x):
    print('Hi i am a function')
    return hello

In [20]:
hello(1)

Hi i am a function


<function __main__.hello>

In [22]:
hello(1)(2)(3) # 이런식으로 사용 가능하다

Hi i am a function
Hi i am a function
Hi i am a function


<function __main__.hello>

### closure
- first class function을 지원하는 언어의 네임 바인딩 기술
- __내부함수를 반환하는 외부함수__ 구조
- 내부 함수를 둘러싼 환경을 계속 유지하다가 해당 내부 함수가 호출될때 다시 꺼내서 사용하는 함수

-> 동적 함수생성을 가능하게 한다

In [40]:
def outer():
    x = 1
    def inner():
        print(x)
    return inner

foo = outer()
foo()

1


[참고 블로그](https://khanrc.tistory.com/entry/decorator%EC%99%80-closure)

foo는 outer의 리턴값인 inner를 받은 함수이다

inner안에서 사용되는 x가 외부에 있기에 foo를 호출하는 시점에서는 x가 존재하지 않아야 한다

그런데 위와 같이 잘 실행된다. 이것이 __function closure__ 이다

function closure는 inner 함수가 정의 될때 그 함수를 감싸고 있는 namespace가 어떻게 생겼는지 기억한다는 의미다

foo.\_\_closure\_\_로 foo 함수를 감싸고 있는 scope의 변수들을 볼 수 있다

In [49]:
print(foo.__name__)
print(foo.__closure__)

inner
(<cell at 0x000001A1C1DE8A38: int object at 0x0000000075FA6DE0>,)


이는 어떠한 함수를 객체로 받을때 그 함수를 감싸는 scope의 변수들 또한 같이 가져간다는 의미다. 따라서 아래와 같이 기능한다

In [54]:
def outer(x):
    def inner():
        print(x)
    return inner

print1 = outer(1)
print2 = outer(2)

In [55]:
print1()

1


In [56]:
print2()

2


## decorator

데코레이터는 __함수를 인자로 받아 꾸며주는__ 기능을 한다.

즉, 함수를 __래핑__ 하는 기능이라 할 수 있다

-> 데코레이터가 붙은 함수의 인자로 데코레이터 아래에 정의된 함수를 넣는다

function closure 개념을 가지고 함수를 꾸며주는 함수를 정의해보자

func을 인자로 받아 func의 앞뒤로 꾸며주는 closure를 정의한다

In [67]:
def verbose(func):
    def new_func(x):
        print('before func')
        func(x)
        print('after func')
    return new_func

def my_func(x):
    print('this is func in which {} is passed'.format(x))

print('without closure:')
my_func(1)
print('----------------------------')
print('with closure:')
verbose(my_func)(1)

without closure:
this is func in which 1 is passed
----------------------------
with closure:
before func
this is func in which 1 is passed
after func


함수를 꾸며주는 구조의 closure를 데코레이터 기호로 표시할 수 있다

In [68]:
def verbose(func):
    def new_func(x):
        print('before func')
        func(x)
        print('after func')
    return new_func

@verbose
def my_func(x):
    print('this is func in which {} is passed'.format(x))
    
my_func(1)

before func
this is func in which 1 is passed
after func


3층구조 데코레이터 -> 데코레이터의 인자를 받는다( @app.route('/') )

In [58]:
def extrafun(hi):
    def decorator(func):
        def wrapper(x):
            print("This is stuff before dec")
            func(x)
            print("This is stuff after dec")
        return wrapper
    return decorator

@extrafun(1)
def newfunc(c):
    print("this is new function", 'passed par is:',c)

newfunc(30)

This is stuff before dec
this is new function passed par is: 30
This is stuff after dec
